## Importacion de librerias

In [4]:
import pandas as pd
import plotly.express as px

## 1) Carga de Datos

In [13]:
try:
    data = pd.read_csv('Ventas.csv', sep =',')
    print("El archivo fue cargado correctamente")
except Exception as err:
    print("El archivo no fue cargado correctamente, error:", err)
    
data.head()

El archivo fue cargado correctamente


,fecha,producto,cantidad,precio_unitario
0,2024-01-01,Producto A,10,5.0
1,2024-01-01,Producto B,5,10.0
2,2024-01-02,Producto A,20,5.0
3,2024-01-02,Producto C,15,7.0
4,2024-01-03,Producto B,10,10.0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fecha            6 non-null      object 
 1   producto         6 non-null      object 
 2   cantidad         6 non-null      int64  
 3   precio_unitario  6 non-null      float64
dtypes: float64(1), int64(1), object(2)
memory usage: 320.0+ bytes


## 2) Limpieza de datos

In [7]:
# Revisamos cuales de la variables tienen valores nulos
for columna in data.columns:
    cantidad_nulos = data[columna].isnull().sum()  
    if cantidad_nulos > 0:
        print(f"Se encontraron {cantidad_nulos} valores nulos en la columna '{columna}'.")
    else:
        print(f"No se encontraron valores nulos en la columna '{columna}'.")

No se encontraron valores nulos en la columna 'fecha'.
No se encontraron valores nulos en la columna 'producto'.
No se encontraron valores nulos en la columna 'cantidad'.
No se encontraron valores nulos en la columna 'precio_unitario'.


In [14]:
# En caso encontremos valores nulos o celdas con un solo espacio reemplazaremos con null
data.replace(r'^\s*$', pd.NA, regex=True, inplace=True)

In [8]:
# En caso se requiere hacer una imputacion. 
# Realizaremos un imputacion por la moda para datos categoricos y por la media por datos numericos en caso existan valores nulos
for columna in data.columns:
    if data[columna].isnull().any():
        if data[columna].dtype == 'object': # datos categoricos
            moda = data[columna].mode()[0] # imputamos con la moda
            data[columna].fillna(moda, inplace=True)
            print(f"Se imputaron los valores nulos en el campo '{columna}' con la moda: {moda}")
        else:  # si es numérico
            media = data[columna].mean() # imputamos con la media
            data[columna].fillna(media, inplace=True)
            print(f"Se imputaron los valores nulos en el campo '{columna}' con la media: {media}")

## 3) Analisis de Datos

In [9]:
# Creamos un campo "Venta" para facilitar los calculos solicitados.
data['Venta'] = data['cantidad'] * data['precio_unitario']
data

,fecha,producto,cantidad,precio_unitario,Venta
0,2024-01-01,Producto A,10,5.0,50.0
1,2024-01-01,Producto B,5,10.0,50.0
2,2024-01-02,Producto A,20,5.0,100.0
3,2024-01-02,Producto C,15,7.0,105.0
4,2024-01-03,Producto B,10,10.0,100.0
5,2024-01-03,Producto C,5,7.0,35.0


### a) Total de ventas por cada producto

In [10]:
# creamos la tabla 'tabla_ventas' por cada producto
print('Total de ventas por cada producto')
tabla_ventas = data.groupby('producto')['Venta'].sum().reset_index()
tabla_ventas.columns = ['producto', 'totalventas']
tabla_ventas

Total de ventas por cada producto


,producto,totalventas
0,Producto A,150.0
1,Producto B,150.0
2,Producto C,140.0


### b) El producto mas vendido

In [11]:
# Buscaremos cual es la mayor venta
venta_mayor = tabla_ventas['totalventas'].max()

# Mostraremos el producto o productos con el mayor monto
productos_encontrados = tabla_ventas[tabla_ventas['totalventas'] == venta_mayor]

print("Se muestra la tabla con el producto o productos más vendidos del mes:\n")
productos_encontrados

Se muestra la tabla con el producto o productos más vendidos del mes:



,producto,totalventas
0,Producto A,150.0
1,Producto B,150.0


### c) Los 5 días con mayor cantidad de ventas en funcion a la cantidad de productos vendidos

In [21]:
# Creamos la tabla 'ventas_por_dia' para agrupar por fecha y cantidad de prodcutos.
ventas_por_dia = data.groupby('fecha')['cantidad'].sum().reset_index()

# Ordenamos los días por la cantidad de productos vendidos, de mayor a menor
ventas_por_dia = ventas_por_dia.sort_values(by='cantidad', ascending=False)

# nombramos las columnas
ventas_por_dia.columns = ['fechaventa', 'n_productos']

# Extraemos las 5 fechas donde se realizo la mayor cantidad de venta
top_5_dias = ventas_por_dia.head(5)

# mostraremos la tabla
print("Las 5 fechas con mayor cantidad de productos vendidos son:\n")
print(top_5_dias)

Las 5 fechas con mayor cantidad de productos vendidos son:

   fechaventa  n_productos
1  2024-01-02           35
0  2024-01-01           15
2  2024-01-03           15


## 4) Visualizacion de Graficos

In [22]:
# apartir de la 'tabla_ventas' ya creada obtendremos el grafico dinamico.
fig = px.bar(tabla_ventas, 
             x='producto', 
             y='totalventas', 
             title="Ventas por Producto", 
             labels={'producto':'Tipo de Producto', 'totalventas':'Total de Ventas (mes en curso)'},
             color='producto',
             text='totalventas')

fig.show()

# Resultado:
# En el grafico podemos observar que los productos lideres en venta del mes en curso fueras los productos A y B